In [1]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from jiwer import wer
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler

2025-03-01 08:56:14.668790: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 08:56:14.716314: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-01 08:56:15.651281: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = torch.device("cuda")

run_number = 0
epochs = 10
batch_size = 16
lr = 5e-5
weight_decay = 0.01

train_dataset_path = "clean_data/train-clean-100.csv"
val_dataset_path = "clean_data/test-clean.csv"

model_name = "google/byt5-small"

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Cuda available and enabled") if torch.cuda.is_available() else None
if device.type == "cuda":
    print(f"Using CUDA with cuDNN Enabled: {torch.backends.cudnn.enabled}")

Cuda available and enabled
Using CUDA with cuDNN Enabled: True


In [3]:
start_epoch = 0
log_dir = f"logs/run_{run_number}"
writer = SummaryWriter(log_dir=log_dir, purge_step=start_epoch)

In [4]:
# Custom dataset class
class G2PDataset(Dataset):
    def preprocess(self):
        self.data["text"] = "grapheme to phoneme: " + self.data["text"]
        self.data["len"] = self.data["text"].str.len() + 1
        self.data = self.data[self.data["len"] < self.max_length]

    def __init__(self, file_path, max_length=512):
        self.data = pd.read_csv(file_path)
        self.max_length = max_length

        print(f"Dataset: {file_path}")
        print(f"Original dataset size:\t{len(self.data)}")
        self.preprocess()
        print(f"Reduced dataset size:\t{len(self.data)}\n")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        grapheme_text = self.data.iloc[idx]["text"]
        phoneme_text = self.data.iloc[idx]["phonemes"]

        return grapheme_text, phoneme_text

# Collate function for dynamic padding
def collate_fn(batch):
    inputs, targets = zip(*batch)

    # Tokenize with dynamic padding (longest in batch)
    input_enc = tokenizer(list(inputs), padding=True, return_tensors="pt", truncation=False)
    target_enc = tokenizer(list(targets), padding=True, return_tensors="pt", truncation=False)

    return {
        "input_ids": input_enc.input_ids,
        "attention_mask": input_enc.attention_mask,
        "labels": target_enc.input_ids,
    }

train_dataset = G2PDataset(train_dataset_path)
val_dataset = G2PDataset(val_dataset_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)

Dataset: clean_data/train-clean-100.csv
Original dataset size:	28538
Reduced dataset size:	28538

Dataset: clean_data/test-clean.csv
Original dataset size:	2620
Reduced dataset size:	2619



In [5]:
# clear gpu_cache
def clear_gpu_cache():
    if device is torch.device("cuda"):
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        torch.backends.cudnn.benchmark = False  # Reset benchmark tuning
        torch.backends.cudnn.benchmark = True  # Re-enable after clearing
    elif device is torch.device("mps"):
        torch.mps.empty_cache()

# Training function
def train_model(model, train_loader, val_loader, epochs, writer, verbose=True):
    model.train()

    for epoch in range(start_epoch, start_epoch + epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{start_epoch + epochs}", leave=True)
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # with torch.autocast("cuda", dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, use_cache=False)
            loss = outputs.loss

            if torch.isnan(loss) or torch.isinf(loss):
                print("Warning: NaN loss detected! Skipping batch.")
                continue

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
            total_loss += loss.item()

            progress_bar.set_postfix(loss=loss.item())
            del input_ids, attention_mask, labels

        avg_train_loss = total_loss / len(train_loader)
        if verbose:
            print(f"Epoch {epoch+1}, Loss: {avg_train_loss:.4f}")

        writer.add_scalar("Training Loss", avg_train_loss, epoch)
        clear_gpu_cache()

        # Validation
        avg_val_loss, exact_match_accuracy, avg_per = validate_model(model, val_loader)
        writer.add_scalar("Validation Loss", avg_val_loss, epoch)
        writer.add_scalar("Exact Match", exact_match_accuracy, epoch)
        writer.add_scalar("Average PER", avg_per, epoch)

        # save the model
        model.save_pretrained(f"weights/weights_{run_number}")
        torch.save(optimizer.state_dict(), f"weights/weights_{run_number}/optimizer.pt")
        # model.save_pretrained(f"weights/weights_{run_number}/epoch_{epoch}")
        # torch.save(optimizer.state_dict(), f"weights/weights_{run_number}/epoch_{epoch}/optimizer.pt")

        clear_gpu_cache()

    return


# Validation function
def validate_model(model, val_loader, verbose=True):
    model.eval()
    total_loss = 0
    total_exact = 0
    total_samples = 0
    total_per = 0

    progress_bar = tqdm(val_loader, desc="Validation", leave=True)

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with torch.autocast("cuda", dtype=torch.float16):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, use_cache=False)
                loss = outputs.loss

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())


            # calculate per
            predicted_ids = torch.argmax(outputs.logits, dim=-1)
            pred_phonemes = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
            true_phonemes = tokenizer.batch_decode(labels, skip_special_tokens=True)

            for pred, target in zip(pred_phonemes, true_phonemes):
                if pred.strip() == target.strip():
                    total_exact += 1
                else:
                    total_per += wer(target, pred)  # WER works similarly for phonemes

            total_samples += len(batch['labels'])

            clear_gpu_cache()

    avg_val_loss = total_loss / len(val_loader)
    exact_match_accuracy = total_exact / total_samples
    avg_per = total_per / total_samples

    if verbose:
        print(f"Validation Loss: {avg_val_loss:.4f}")
        print(f"Validation Accuracy (Exact Match): {exact_match_accuracy * 100:.2f}%")
        print(f"Average Phoneme Error Rate (PER): {avg_per * 100:.2f}%")

    return avg_val_loss, exact_match_accuracy, avg_per

In [6]:
train_model(model, train_loader, val_loader, epochs, writer)

writer.close()

Epoch 1/10:   0%|          | 2/1784 [00:00<06:41,  4.43it/s]

Epoch 1/10:   0%|          | 5/1784 [00:00<04:59,  5.95it/s]

Epoch 1/10:   0%|          | 7/1784 [00:01<04:05,  7.22it/s]

Epoch 1/10:   1%|          | 9/1784 [00:01<03:34,  8.26it/s]

Epoch 1/10:   1%|          | 11/1784 [00:01<03:25,  8.64it/s]

...

Epoch 1/10:   4%|▍         | 69/1784 [00:08<03:29,  8.18it/s]

Epoch 1/10:   4%|▍         | 71/1784 [00:08<03:12,  8.91it/s]

Epoch 1/10:   4%|▍         | 74/1784 [00:09<02:59,  9.51it/s]

Epoch 1/10:   4%|▍         | 76/1784 [00:09<03:31,  8.06it/s]

KeyboardInterrupt: 